<a href="https://colab.research.google.com/github/sunshineluyao/UTXO/blob/main/Partition_the_data_by_spent_year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


run the following code in big query and generate joint_date
reference: https://cloud.google.com/bigquery/docs/reference/standard-sql/timestamp_functions
```
SELECT 
  value,
  FORMAT_TIMESTAMP("%Y-%m-%d", block_timestamp) AS block_date,
  FORMAT_TIMESTAMP("%Y-%m-%d", spent_block_timestamp) AS spent_block_date,
  EXTRACT(YEAR FROM block_timestamp) AS year,
  EXTRACT(Month FROM block_timestamp) AS Month
FROM `crypto-291811.UTXO.joint_all`
```



Create Partition Table
https://fivetran.com/docs/destinations/bigquery/partition-table

run the following to create dateset partition by the spent year, do the same for all block years until 2020


```
SELECT 
  value,
  block_date,
  spent_block_date
FROM `crypto-291811.UTXO.joint_date` 
WHERE year=2009 or year=2010
```




In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import bigquery

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'crypto-291811'
client = bigquery.Client(project=project_id)

for dataset in client.list_datasets():
  print(dataset.dataset_id)

UTXO


Now, we can first deal with the data that has creation year in 2009 or 2010, Note that the data is smaller than 1GB so that we can import directly to colab. 
After we deal with the data, we need to delete all data that has been spent up to 2010. 

In [3]:
###To retrieve table rows as a pandas.DataFrame:
# from google.cloud import bigquery
# client = bigquery.Client()
project = "crypto-291811"
dataset_id = "UTXO"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("joint_2010")
table = client.get_table(table_ref)

df_2010 = client.list_rows(table).to_dataframe()
df_2010.head()

,value,block_date,spent_block_date
0,5000000000,2009-01-03,None
1,5000000000,2009-01-09,None
2,5000000000,2009-01-09,None
3,5000000000,2009-01-09,None
4,5000000000,2009-01-09,None


In [4]:
df_2010.tail()

,value,block_date,spent_block_date
137520,5000000000,2010-10-11,2020-10-11
137521,5000000000,2010-10-13,2020-10-11
137522,5000000000,2010-11-10,2020-10-11
137523,5000000000,2010-11-09,2020-10-14
137524,5000000000,2010-09-23,2020-10-17


In [8]:
###To retrieve table rows as a pandas.DataFrame:
# from google.cloud import bigquery
# client = bigquery.Client()
project = "crypto-291811"
dataset_id = "UTXO"

dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table("joint_2011")
table = client.get_table(table_ref)

df_2011 = client.list_rows(table).to_dataframe()
df_2011.head()

,value,block_date,spent_block_date
0,5475757,2011-09-01,2012-02-09
1,344000000,2011-09-05,2011-09-06
2,12508787,2011-05-30,2011-06-06
3,88999695,2011-08-30,2011-09-10
4,40894346,2011-11-11,2012-05-03


In [9]:
### sort the data by block_date
df_2010.dtypes

value               object
block_date          object
spent_block_date    object
dtype: object

In [10]:
import pandas as pd

In [11]:
df_2010['block_date']=pd.to_datetime(df_2010['block_date'])
df_2010['spent_block_date']=pd.to_datetime(df_2010['spent_block_date'])

In [12]:
df_2010.dtypes

value                       object
block_date          datetime64[ns]
spent_block_date    datetime64[ns]
dtype: object

In [13]:
df_2011['block_date']=pd.to_datetime(df_2011['block_date'])
df_2011['spent_block_date']=pd.to_datetime(df_2011['spent_block_date'])

In [14]:
df_2010=df_2010.sort_values(by=['block_date','spent_block_date'])
df_2010.head()

,value,block_date,spent_block_date
0,5000000000,2009-01-03,NaT
21553,5000000000,2009-01-09,2009-01-12
1,5000000000,2009-01-09,NaT
2,5000000000,2009-01-09,NaT
3,5000000000,2009-01-09,NaT


In [15]:
df_2011=df_2011.sort_values(by=['block_date','spent_block_date'])
df_2011.head()

,value,block_date,spent_block_date
15563,957000000,2011-01-01,2011-01-02
46791,29000000,2011-01-01,2011-01-02
64685,33895000000,2011-01-01,2011-01-02
94558,126000000,2011-01-01,2011-01-02
113788,4552000000,2011-01-01,2011-01-02


In [16]:
df_2010.tail()

,value,block_date,spent_block_date
136385,5000000,2010-12-31,2017-07-30
136386,5000000,2010-12-31,2017-07-30
136654,523000000,2010-12-31,2017-10-16
137004,5000000,2010-12-31,2017-12-15
137401,5000000,2010-12-31,2019-10-16


In [17]:
df_2011.tail()

,value,block_date,spent_block_date
577463,71332333,2011-12-31,2019-12-17
153925,71176732,2011-12-31,2020-03-21
123856,103000000,2011-12-31,2020-04-19
1161818,100000,2011-12-31,2020-04-26
1156511,100000,2011-12-31,2020-05-11


In [18]:
# Importing drive method from colab for accessing google drive
from google.colab import drive

In [19]:
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
df_2010.to_csv('/content/drive/My Drive/UTXO/joint_2010.csv')
df_2011.to_csv('/content/drive/My Drive/UTXO/joint_2011.csv')